# Feature Extraction Verification

This notebook audits the `src/features/feature_extraction.py` module. 
We verified the pipeline in the previous notebook. Now we focus on:
1. **EDA**: Visualizing Phasic/Tonic decomposition and SCR peak detection.
2. **ECG**: Verifying R-peak detection on the downsampled (35Hz) signal.
3. **Features**: Checking if the extracted features (HRV, Slope, etc.) show separability between Baseline and Stress states.

In [ ]:
# 1. Import Libraries & Load Configuration
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.stats as stats

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.config import load_config
from src.data.make_dataset import WESADPreprocessor
from src.features.feature_extraction import FeatureExtractor
from src.visualization.plots import set_plot_style

set_plot_style()
config = load_config()
print("Configuration Loaded.")

In [ ]:
# 2. Load Preprocessed Data (Subject S2)
preprocessor = WESADPreprocessor()
subject_id = 'S2'

print(f"Loading data for {subject_id}...")
windows = preprocessor.process_subject(subject_id)
print(f"Successfully loaded {len(windows)} windows.")

# Basic check
print("Sample Window Keys:", windows[0].keys())

## 3. Audit EDA Processing
We verify the logic used in `FeatureExtractor._process_eda`. 
We replicate the logic here to visualize the Phasic vs Tonic split and SCR peaks.

In [ ]:
# Find a window with significant EDA activity (high range)
eda_ranges = [np.max(w['EDA']) - np.min(w['EDA']) for w in windows]
audit_idx = np.argmax(eda_ranges)
window = windows[audit_idx]
print(f"Auditing Window Index: {audit_idx}")

eda_signal = np.array(window['EDA'])
fs = 35 # Target sampling rate

# --- Replicate FeatureExtractor Logic ---
# 1. Butterworth Low-pass filter for Tonic
b, a = signal.butter(3, 0.05 / (0.5 * fs), btype='low')
tonic = signal.filtfilt(b, a, eda_signal)
phasic = eda_signal - tonic

# 2. Detect SCR Peaks in Phasic component
peaks, _ = signal.find_peaks(phasic, height=0.01, distance=fs)

# --- Visualization ---
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
time = np.arange(len(eda_signal)) / fs

# Plot 1: Decomposition
axes[0].plot(time, eda_signal, label='Raw EDA')
axes[0].plot(time, tonic, label='Tonic (Low-pass)', linestyle='--', linewidth=2)
axes[0].set_title("EDA Decomposition: Raw vs Tonic")
axes[0].legend()
axes[0].set_ylabel("uS")

# Plot 2: Phasic & Peaks
axes[1].plot(time, phasic, color='green', label='Phasic (Residual)')
axes[1].plot(time[peaks], phasic[peaks], "rx", markersize=10, label=f'Detected SCRs (n={len(peaks)})')
axes[1].axhline(0.01, color='gray', linestyle='--', alpha=0.5, label='Threshold (0.01 uS)')
axes[1].set_title("Phasic Component & SCR Events")
axes[1].legend()
axes[1].set_ylabel("uS")

# Plot 3: Context (Motion)
axes[2].plot(time, window['ACC_x'], color='gray', alpha=0.5, label='Motion (ACC X)')
axes[2].set_title("Motion Context")
axes[2].set_xlabel("Time (s)")
axes[2].legend()

plt.tight_layout()
plt.show()

## 4. Audit ECG Processing
Verify if the simple bandpass + peak detection works reasonably well on the 35Hz downsampled ECG.

In [ ]:
ecg_signal = np.array(window['ECG'])

# --- Replicate Logic ---
# Bandpass 1-15Hz
b, a = signal.butter(3, [1.0 / (0.5 * fs), 15.0 / (0.5 * fs)], btype='band')
clean_ecg = signal.filtfilt(b, a, ecg_signal)
# Detect R-peaks
peaks, _ = signal.find_peaks(clean_ecg, height=np.mean(clean_ecg), distance=int(0.4 * fs))

# --- Visualization ---
plt.figure(figsize=(12, 4))
time = np.arange(len(ecg_signal)) / fs

plt.plot(time, clean_ecg, label='Processed ECG (1-15Hz)')
plt.plot(time[peaks], clean_ecg[peaks], "ro", label='Detected R-Peaks')
plt.axhline(np.mean(clean_ecg), color='gray', linestyle='--', label='Detection Threshold')

est_bpm = len(peaks) * (60 / (len(ecg_signal)/fs))
plt.title(f"ECG Audit: Found {len(peaks)} beats (~{est_bpm:.0f} BPM)")
plt.xlabel("Time (s)")
plt.legend()
plt.show()

## 5. Generate Full Feature Vector
Run the `FeatureExtractor` on a single window to see the output format.

In [ ]:
extractor = FeatureExtractor(config)
features = extractor.extract_features(window)

print("--- Extracted Features ---")
feat_series = pd.Series(features)
print(feat_series)

# Check for nans
if feat_series.isnull().any():
    print("\nWARNING: NaNs detected!")
else:
    print("\nData Integrity Check: Pass (No NaNs)")

## 6. Feature Separability Analysis
Compare 'Baseline' (1) vs 'Stress' (2) for Subject S2.
We extract features for a subset of windows and verify if we see distributional differences.

In [ ]:
# Limit to first 500 windows to save time during interactive audit
limit = 500
audit_windows = windows[:limit]

data = []
for w in audit_windows:
    feats = extractor.extract_features(w)
    feats['label'] = w['label']
    data.append(feats)

df = pd.DataFrame(data)

# Filter for Baseline (1) and Stress (2)
df_filtered = df[df['label'].isin([1, 2])].copy()
df_filtered['label_name'] = df_filtered['label'].map({1: 'Baseline', 2: 'Stress'})

print(f"Analyzed {len(df_filtered)} windows (Baseline vs Stress).")

# Plot Boxplots for key features
plot_cols = ['eda_mean', 'eda_acc_scr_count', 'ecg_hr_bpm', 'ecg_rmssd', 'temp_slope']

fig, axes = plt.subplots(1, len(plot_cols), figsize=(20, 5))

for i, col in enumerate(plot_cols):
    if col in df_filtered.columns:
        # Use pandas built-in boxplot for simplicity
        df_filtered.boxplot(column=col, by='label_name', ax=axes[i], grid=False)
        axes[i].set_title(col)
        axes[i].set_xlabel("")
        axes[i].set_ylabel("Value")

plt.suptitle("Feature Separability (Baseline vs Stress) - S2 Audit")
plt.tight_layout()
plt.show()

## 7. Conclusion
The feature extraction logic appears robust visually. The boxplots in Section 6 provide a preliminary confirmation that features like EDA Mean and Temp Slope differ between stress and baseline states. The next step is to run this pipeline on the full dataset.